# Ngio

In [1]:
from pathlib import Path
from tifffile import TiffFile
from ngio import create_empty_ome_zarr
from dask.array import Array
import time

timer = time.perf_counter()
path = Path("data") / "Human_neuron.tif"

with TiffFile(path) as tif:
    shape = tif.series[0].shape
    dtype = str(tif.series[0].dtype)
    
print(f"Read TIFF metadata in {time.perf_counter() - timer:.2f} seconds")


timer = time.perf_counter()
zarr_store = Path("data") / "Human_neuron_ngio.zarr"
ome_zarr = create_empty_ome_zarr(store=zarr_store, 
                                 xy_pixelsize=0.24,
                                 z_spacing=0.29, 
                                 shape=shape,
                                 dtype=dtype, 
                                 chunks=(1, shape[1], shape[2]),
                                 overwrite=True)
image = ome_zarr.get_image()
print(f"Created empty OME-Zarr in {time.perf_counter() - timer:.2f} seconds")

timer = time.perf_counter()
with TiffFile(path) as tif:
    for z, page in enumerate(tif.series[0].pages):
        assert page is not None
        plane = page.asarray()[None, ...]
        image.set_array(plane, z=z)
        
print(f"Written {shape[0]} planes in {time.perf_counter() - timer:.2f} seconds")

timer = time.perf_counter()
dask_array = image.get_array(mode="dask")
assert isinstance(dask_array, Array)
dask_array = dask_array.rechunk((128, 128, 128))
image.set_array(dask_array)
print(f"Rechunked array in {time.perf_counter() - timer:.2f} seconds")

timer = time.perf_counter()
image.consolidate()
print(f"Consolidated array in {time.perf_counter() - timer:.2f} seconds")

timer = time.perf_counter()
ome_zarr.set_channel_percentiles()
print(f"Set channel percentiles in {time.perf_counter() - timer:.2f} seconds")

Read TIFF metadata in 0.05 seconds
Created empty OME-Zarr in 0.55 seconds
Written 527 planes in 2.50 seconds


KeyboardInterrupt: 